In [ ]:
from lmfit import *

In [ ]:
truths = (0.3, 0.0, 70)

x = np.array(zcmb)
np.random.seed(0)
y = model_equation(x, *truths)+0.1*np.random.randn(x.size)
yerr = np.array(dmb)


model = lmfit.Model(model_equation)
p = model.make_params()
p.add('OM', value=0.5, min=0, max=1)
p.add('OL', value=0.5, min=0, max=1)
p.add('H0', value=70, min=0, max=1000)
print(p)
result = model.fit(data=y, params=p, x=x, method='Nelder', nan_policy='omit')

report = lmfit.report_fit(result)
print(report)
result.plot()
plt.xscale('log')
plt.show()

emcee_kws = dict(steps=5000, burn=500, thin=20, is_weighted=False,
                 progress=True)
emcee_params = model.make_params()
emcee_params.add('OM', value=result.params['OM'].value, min=0, max=0.5)
emcee_params.add('OL', value=result.params['OL'].value, min=0, max=1)
emcee_params.add('H0', value=result.params['H0'].value, min=0, max=1000)
emcee_params.add('__lnsigma', value=log10(0.1), min=log10(0.001), max=log10(2.0))

result_emcee = model.fit(data=y, x=x, params=emcee_params, method='emcee',
                         nan_policy='omit', fit_kws=emcee_kws)

emcee_report = lmfit.report_fit(result_emcee)
print(emcee_report)

result_emcee.plot()
plt.xscale('log')
plt.show()
result_emcee.plot_fit()
plt.plot(x, model.eval(params=result.params, x=x), '--', label='emcee')
plt.legend()
plt.xscale('log')
plt.show()

emcee_corner = corner.corner(result_emcee.flatchain, labels=result_emcee.var_names,
                             truths=list(result_emcee.params.valuesdict().values()))
plt.show()